In [2]:
from common import *
import xml.etree.ElementTree as ET

Using TensorFlow backend.


In [9]:
fname = join(join(DATA_FOLDER, 'rawdata'), 'all_docs_raw.txt')

In [10]:
# all_docs = glob(join(DATA_FOLDER, 'rawdata') + '/**/*.xml', recursive=True)
# all_docs = sorted(all_docs)
# with open(fname, mode='wt', encoding='utf-8') as f:
#     f.write('\n'.join(all_docs))

In [11]:
with open(fname, mode='r', encoding='utf-8') as f:
    all_docs = f.read().splitlines()

In [12]:
len(all_docs)

1266990

In [268]:
afile = '../data/rawdata/2016/RU/A/20160110/2014114983/2014114983.xml'
c1file = '../data/rawdata/2016/RU/C/20160827/0001804260/0001804260.xml'
c2file = '../data/rawdata/2003/C2/20030120/0002196401/0002196401.xml'
ufile = '../data/rawdata/2007/RU/U1/20070327/0000061980/0000061980.xml'

In [212]:
def get_node(root, name, lang='ru'):
    for node in root.iter(name):
        if node.attrib['lang'].lower() == lang:
            return node

In [318]:
doc = {}

tree = ET.parse(c2file)
root = tree.getroot()

doc.update(root.attrib)

SDOBI = get_node(root, 'SDOBI')
name = SDOBI.find('./B500/ru-b540/ru-b542').text.lower()
doc['name_raw'] = name

node = get_node(root, 'abstract')
if node is not None:
    doc['abstract_raw'] = ' '.join(node.itertext()).lower()

node = SDOBI.find('./B500/B560/ru-b560')
if node is not None:
    doc['ru-b560'] = ' '.join(node.itertext()).lower()

node = get_node(root, 'description')
if node is not None:
    doc['description_raw'] = ' '.join(node.itertext()).lower()

node = get_node(root, 'claims')
if node is not None:
    doc['claims_raw'] = ' '.join(node.itertext()).lower()
            

In [319]:
print(doc['abstract_raw'])

агромост содержит ферму, раму с культиваторными ножами, закрепленными на рычагах с возможностью колебательного движения, и культиваторные лемеха с двусторонней заточкой, нить с выполненными 

на ней реперами, взаимодействующую с датчиком, и штанги. агромост имеет подвижную платформу, на которой закреплен датчик. нить с реперами закреплена на ферме. штанги выполнены с талрепами для 

поочередного подъема сторон рамы над колеями и регулирования их высотного положения относительно уровня почвы. сокращается число требуемых сельхозорудий, упрощается конструкция агромоста и снижается 

его стоимость. 4 ил.


In [320]:
doc

{'abstract_raw': 'агромост содержит ферму, раму с культиваторными ножами, закрепленными на рычагах с возможностью колебательного движения, и культиваторные лемеха с двусторонней заточкой, нить с выполненными \n\nна ней реперами, взаимодействующую с датчиком, и штанги. агромост имеет подвижную платформу, на которой закреплен датчик. нить с реперами закреплена на ферме. штанги выполнены с талрепами для \n\nпоочередного подъема сторон рамы над колеями и регулирования их высотного положения относительно уровня почвы. сокращается число требуемых сельхозорудий, упрощается конструкция агромоста и снижается \n\nего стоимость. 4 ил.',
 'claims_raw': ' агромост, содержащий ферму, раму с культиваторными ножами, закрепленными на рычагах с возможностью \nколебательного \nдвижения в направлении, перпендикулярном направлению рядков растений, и с культиваторными лемехами с двусторонней заточкой, нить с выполненными на ней реперами, взаимодействующую с \nдатчиком, \nвспомогательное оборудование и штанг